In [1]:
import argparse
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import pickle
from skimage.transform import resize
import sys
import torch
import torchvision
from torchvision.transforms import transforms as transforms

sys.path.append('..')
from utils import Pose

%matplotlib inline

In [2]:
# initialize the model
model = torchvision.models.detection.keypointrcnn_resnet50_fpn(
    pretrained=True, num_keypoints=17
)
# set the computation device
# device = torch.device(0)
device = torch.device('cpu')

# load the model on to the computation device and set to eval mode
model.to(device).eval()

# transform to convert the image to tensor
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
# image_files = sorted(glob.glob('../data/raw/*.jpg'))

out_dir = '../data/figures/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

# for i, fname in enumerate(image_files):
i = 0

while True:
    # read metatdata, one row at a time
    fname = pd.read_csv(
        '../data/raw/metadata.csv',
        delimiter=';',
        on_bad_lines=lambda x: [x[0], ';'.join(x[1:])],
        engine='python',
        skiprows=i,
        nrows=1
    ).iloc[0, 0]
    sys.stdout.write('\r\t%d\t' % i)
    
    try:
        image = Image.open(fname).convert('RGB')

        # NumPy copy of the image for OpenCV functions
        img = np.array(image, dtype=np.float32) / 255

        # transform the image
        image = transform(image)

        # add a batch dimension
        image = image.unsqueeze(0).to(device)

        # run keypoint detection
        with torch.no_grad():
            output = model(image)[0]
        output['filename'] = fname
        
        H, W, _ = img.shape

        boxes = output['boxes'].cpu()
        scores = output['scores'].detach().cpu().numpy()

        for j, b in enumerate(boxes):
            if scores[j] < 0.9:
                continue
            if output['keypoints_scores'][j].min() < 0:
                continue
            
            p = Pose(index=j, **output)
            p.resize()
            
            outfile = out_dir + fname.split('/')[-1].rstrip('.jpg') + '_%04d' % j + '.pkl'
            with open(outfile, 'wb') as fout:
                pickle.dump(p, fout)
            
    except Exception as e:
        print(e)
    
    i += 1

	0	

/home/alan/miniconda3/envs/mytorch/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272204863/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


	52	[Errno 2] No such file or directory: '../data/raw/orc_barbarian_male_000000053.jpg'
	61	

/home/alan/miniconda3/envs/mytorch/lib/python3.9/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


	117	

/home/alan/miniconda3/envs/mytorch/lib/python3.9/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


	128	

In [ ]:
p = Pose(**output)

In [ ]:
p.plot()

In [ ]:
p.resize()
p.plot()

In [ ]:
plt.imshow(p.Z)

In [ ]:
pd.read_csv?